In [1]:
import os
import sys
os.chdir(os.getcwd())
pydice_folder = os.path.dirname(os.getcwd()) + "\\06_Code"
sys.path.insert(1, pydice_folder)
pydice_folder

'e:\\Year_2_Quarter_4\\Thesis\\06_Code'

In [2]:
# Introdction to the code

### Importing all the nessesarry libraries
 

from __future__ import (unicode_literals, print_function, absolute_import,
                        division)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')


In [3]:
#Importing the needed Ema workbench libraries

from ema_workbench import (Model, Policy, CategoricalParameter,ScalarOutcome, IntegerParameter, RealParameter, optimize, Scenario)

from ema_workbench import ema_logging, MultiprocessingEvaluator, SequentialEvaluator

from ema_workbench.em_framework.optimization import EpsilonProgress, HyperVolume

from ema_workbench import (MultiprocessingEvaluator, ema_logging, perform_experiments, Constant)

from ema_workbench.em_framework.salib_samplers import get_SALib_problem

from ema_workbench.em_framework.samplers import sample_levers, sample_uncertainties

from ema_workbench.em_framework.evaluators import LHS, SOBOL, MORRIS, SequentialEvaluator, BaseEvaluator

from SALib.analyze import sobol

from ema_workbench.analysis import parcoords

from ema_workbench.util import ema_logging

from mpl_toolkits.mplot3d import Axes3D  

ema_logging.log_to_stderr(ema_logging.INFO)

from ema_workbench import save_results, load_results



C:\Users\shrid\Anaconda3\lib\site-packages\ema_workbench\analysis\prim.py:31: ImportWarning: altair based interactive inspection not available
  "inspection not available"), ImportWarning)
C:\Users\shrid\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


In [4]:
# Load model
from MyDICE_v2 import PyDICE
dice_sm = Model('EMA', function= Model)


In [5]:
# Specify inputs
dice_sm.uncertainties = [ IntegerParameter('t2xco2_index', 0, 999),
                            IntegerParameter('t2xco2_dist',0 , 2),
                            # IntegerParameter('fdamage', 0, 2),
                            RealParameter('tfp_gr', 0.07, 0.09),
                            RealParameter('sigma_gr', -0.012, -0.008),
                            # RealParameter('emdd', 1.0, 2.0)
                            RealParameter('pop_gr', 0.1, 0.15),
                            RealParameter('fosslim',  4000.0, 13649),
                            IntegerParameter('cback', 100, 600)]

dice_sm.levers = [RealParameter('sr', 0.1, 0.5),
                      RealParameter('prtp_con',  0.001, 0.015),
                      RealParameter('prtp_dam',  0.001, 0.015),
                    #   RealParameter('emuc', 1.0, 2.0),
                      
                      IntegerParameter('periodfullpart', 10, 58),
                      IntegerParameter('miu_period', 10, 58)]


In [6]:
dice_sm.outcomes = [ScalarOutcome('Damages', ScalarOutcome.INFO),
                        ScalarOutcome('Disutility of Damage', ScalarOutcome.INFO),
                        ScalarOutcome('Total Output', ScalarOutcome.INFO),
                        ScalarOutcome('Welfare', ScalarOutcome.INFO)
                        ]


In [7]:
   
# 1. Open Exploration"

#Choose problem formuation
# model, planning_steps = get_model_for_problem_formulation(EMA) ### 2"

### 1.1. Running model with random scenarios and policies

#We start with running the model with 10 random generated policies with 250 scenarios each. By doing this the behavior of the model can be examineted. "



In [8]:
with MultiprocessingEvaluator(dice_sm, n_processes= 8) as evaluator:
    results = evaluator.perform_experiments(scenarios=10, policies=10)


[MainProcess/INFO] pool started
[MainProcess/INFO] performing 10 scenarios * 10 policies * 1 model(s) = 100 experiments


In [ ]:
  



from ema_workbench.analysis import pairs_plotting

fig, axes = pairs_plotting.pairs_scatter(experiments_open, outcomes_open, group_by='policy',
                                         legend=True)
fig.set_size_inches(20,20)
plt.show()


In [ ]:
  ### 1.2. PRIM
    
  #PRIM is used to see which variables are influencing the results the most. By selecting interesting cases out off all cases by giving one or more restrains. We do this by selecting all cases with \"Expected Annual Damage\" is lower than 0.4 and later on also by selecting all cases with Expected number of deaths lower than 0.01."
from ema_workbench.analysis import prim
from ema_workbench.analysis import scenario_discovery_util as sdutil
from sklearn import preprocessing 

x = experiments_open
y = outcomes_open['Expected Annual Damage'] < 0.4
prim_alg = prim.Prim(x, y, threshold=0.8, peel_alpha = 0.1)
box1 = prim_alg.find_box()

box1.show_tradeoff()
plt.show()

In [ ]:
y = np.amax(outcomes)

In [ ]:
#box1.show_tradeoff()
#plt.show()


In [ ]:
#box1.inspect(4)
#box1.inspect(4, style='graph')
#plt.show()
   

In [ ]:
#box1.select(5)
#fig = box1.show_pairs_scatter(4)
#fig.set_size_inches((12,12))
#plt.show()

In [ ]:
x = experiments_open
y = outcomes_open['Expected Number of Deaths'] < 0.005
prim_alg = prim.Prim(x, y, threshold=0.8, peel_alpha = 0.1)
box2 = prim_alg.find_box()

box2.show_tradeoff()
plt.show()

In [ ]:
#box2.inspect(5)
#box2.inspect(5, style='graph')
#plt.show()

In [ ]:
#box1.select(5)
#fig = box2.show_pairs_scatter(5)
#fig.set_size_inches((12,12))

In [ ]:
# 2. Base Case

#To see which uncertaincies have the most influence in the current situation the base case is evaluated. This has been done by setting all policy levers at 0 (No measure taken). Then the model has been runned with a random sampled uncertaincy space (Sobol is used for doing so). After that the influence of the uncairtainties on the indicators are visualised.

# checking policy levers
list(model.levers)

# Initialization for SALib
uncertainty_problem = get_SALib_problem(model.uncertainties)


In [ ]:
### 2.1 Defining Zero Policy Scenarios

#To define the zero plicy scenario we put all policy levers on 0. By doing this we can evaluate what will happens when we are not doing anything in the upcoming years. This will show us if doing nothing is an option or that real action is needed to prevent floods on the IJssel river.

# Choose problem formuation
model, planning_steps = get_model_for_problem_formulation(2)

zero = {'DikeIncrease 0': 0, 'DikeIncrease 1': 0, 'DikeIncrease 2': 0, 'RfR 0': 0, 'RfR 1': 0, 'RfR 2': 0}
zero_pol = {}

for key in model.levers:
    s1, s2 = key.name.split('_')
    zero_pol.update({key.name: zero[s2]})
zero_pol = Policy('Policy 0', **zero_pol)

In [ ]:
# Number of uncertainties in the analysis
n_uncertainties_sobol = 10 ### 100"

#Running the analysis 
with SequentialEvaluator(model) as evaluator:
    results_sobol = evaluator.perform_experiments(n_uncertainties_sobol, zero_pol, uncertainty_sampling=SOBOL, levers_sampling=SOBOL)

# Splitting the experiments from the outcomes of the result
experiments_sobol, outcomes_sobol = results_sobol

# Show which outcomes are measured 
print(outcomes.keys())

In [ ]:
### 2.2. Seperating Outcomes of Interest

#Disaggregated for location (3)
A1_Total_Costs = outcomes_sobol['A.1 Total Costs']
A1_Expected_Number_of_Deaths = outcomes_sobol['A.1_Expected Number of Deaths']
A2_Total_Costs = outcomes_sobol['A.2 Total Costs']
A2_Expected_Number_of_Deaths= outcomes_sobol['A.2_Expected Number of Deaths']
A3_Total_Costs = outcomes_sobol['A.3 Total Costs']
A3_Expected_Number_of_Deaths= outcomes_sobol['A.3_Expected Number of Deaths']
A4_Total_Costs = outcomes_sobol['A.4 Total Costs']
A4_Expected_Number_of_Deaths= outcomes_sobol['A.4_Expected Number of Deaths']
A5_Total_Costs = outcomes_sobol['A.5 Total Costs']
A5_Expected_Number_of_Deaths= outcomes_sobol['A.5_Expected Number of Deaths']

#Aggregated (2)
Expected_Annual_Damage = outcomes_sobol['Expected Annual Damage']
Expected_Number_of_Deaths = outcomes_sobol['Expected Number of Deaths']

In [ ]:
# Change the outcomes to Z-scores by using the sobol analze option
A1_Total_Costs_Score = sobol.analyze(uncertainty_problem, A1_Total_Costs, calc_second_order=True, print_to_console=False)
A1_Expected_Number_of_Deaths_Score = sobol.analyze(uncertainty_problem, A1_Expected_Number_of_Deaths, calc_second_order=True, print_to_console=False)

A2_Total_Costs_Score = sobol.analyze(uncertainty_problem, A2_Total_Costs, calc_second_order=True, print_to_console=False)
A2_Expected_Number_of_Deaths_Score = sobol.analyze(uncertainty_problem, A2_Expected_Number_of_Deaths, calc_second_order=True, print_to_console=False)

A3_Total_Costs_Score = sobol.analyze(uncertainty_problem, A3_Total_Costs, calc_second_order=True, print_to_console=False)
A3_Expected_Number_of_Deaths_Score = sobol.analyze(uncertainty_problem, A3_Expected_Number_of_Deaths, calc_second_order=True, print_to_console=False)

A4_Total_Costs_Score = sobol.analyze(uncertainty_problem, A4_Total_Costs, calc_second_order=True, print_to_console=False)
A4_Expected_Number_of_Deaths_Score = sobol.analyze(uncertainty_problem, A4_Expected_Number_of_Deaths, calc_second_order=True, print_to_console=False)

A5_Total_Costs_Score = sobol.analyze(uncertainty_problem, A5_Total_Costs, calc_second_order=True, print_to_console=False)
A5_Expected_Number_of_Deaths_Score = sobol.analyze(uncertainty_problem, A5_Expected_Number_of_Deaths, calc_second_order=True, print_to_console=False)

Expected_Annual_Damage_Score = sobol.analyze(uncertainty_problem, Expected_Annual_Damage, calc_second_order=True, print_to_console=False)
Expected_Number_of_Deaths_Score = sobol.analyze(uncertainty_problem, Expected_Number_of_Deaths, calc_second_order=True, print_to_console=False)"

In [ ]:
### 2.3 Filtering the Scores"

A1_Total_Costs_Score_filtered = {k:A1_Total_Costs_Score[k] for k in ['ST','ST_conf','S1','S1_conf']}
A1_Expected_Number_of_Deaths_Score_filtered = {k:A1_Expected_Number_of_Deaths_Score[k] for k in ['ST','ST_conf','S1','S1_conf']}

A2_Total_Costs_Score_filtered = {k:A2_Total_Costs_Score[k] for k in ['ST','ST_conf','S1','S1_conf']}
A2_Expected_Number_of_Deaths_Score_filtered = {k:A2_Expected_Number_of_Deaths_Score[k] for k in ['ST','ST_conf','S1','S1_conf']}

A3_Total_Costs_Score_filtered = {k:A3_Total_Costs_Score[k] for k in ['ST','ST_conf','S1','S1_conf']}
A3_Expected_Number_of_Deaths_Score_filtered = {k:A3_Expected_Number_of_Deaths_Score[k] for k in ['ST','ST_conf','S1','S1_conf']}

A4_Total_Costs_Score_filtered = {k:A4_Total_Costs_Score[k] for k in ['ST','ST_conf','S1','S1_conf']}
A4_Expected_Number_of_Deaths_Score_filtered = {k:A4_Expected_Number_of_Deaths_Score[k] for k in ['ST','ST_conf','S1','S1_conf']}

A5_Total_Costs_Score_filtered = {k:A5_Total_Costs_Score[k] for k in ['ST','ST_conf','S1','S1_conf']}
A5_Expected_Number_of_Deaths_Score_filtered = {k:A5_Expected_Number_of_Deaths_Score[k] for k in ['ST','ST_conf','S1','S1_conf']}

Expected_Annual_Damage_Score_filtered = {k:Expected_Annual_Damage_Score[k] for k in ['ST','ST_conf','S1','S1_conf']}
Expected_Number_of_Deaths_Score_filtered = {k:Expected_Number_of_Deaths_Score[k] for k in ['ST','ST_conf','S1','S1_conf']}\n"

In [ ]:
### 2.4. Creating data frame"

A1_Total_Costs_Score_df = pd.DataFrame(A1_Total_Costs_Score_filtered, index=uncertainty_problem['names'])
A1_Expected_Number_of_Deaths_Score_df = pd.DataFrame(A1_Expected_Number_of_Deaths_Score_filtered, index=uncertainty_problem['names'])

A2_Total_Costs_Score_df = pd.DataFrame(A2_Total_Costs_Score_filtered, index=uncertainty_problem['names'])
A2_Expected_Number_of_Deaths_Score_df = pd.DataFrame(A2_Expected_Number_of_Deaths_Score_filtered, index=uncertainty_problem['names'])

A3_Total_Costs_Score_df = pd.DataFrame(A3_Total_Costs_Score_filtered, index=uncertainty_problem['names'])
A3_Expected_Number_of_Deaths_Score_df = pd.DataFrame(A3_Expected_Number_of_Deaths_Score_filtered, index=uncertainty_problem['names'])

A4_Total_Costs_Score_df = pd.DataFrame(A4_Total_Costs_Score_filtered, index=uncertainty_problem['names'])
A4_Expected_Number_of_Deaths_Score_df = pd.DataFrame(A4_Expected_Number_of_Deaths_Score_filtered, index=uncertainty_problem['names'])

A5_Total_Costs_Score_df = pd.DataFrame(A5_Total_Costs_Score_filtered, index=uncertainty_problem['names'])
A5_Expected_Number_of_Deaths_Score_df = pd.DataFrame(A5_Expected_Number_of_Deaths_Score_filtered, index=uncertainty_problem['names'])

Expected_Annual_Damage_Score_df = pd.DataFrame(Expected_Annual_Damage_Score_filtered, index=uncertainty_problem['names'])
Expected_Number_of_Deaths_Score_df = pd.DataFrame(Expected_Number_of_Deaths_Score_filtered, index=uncertainty_problem['names'])"

df_Sobol = pd.DataFrame(data=None, index=Expected_Number_of_Deaths_Score_df.index, columns=['A.1 Total Costs', 'A.1_Expected Number of Deaths', 'A.2 Total Costs', 'A.2_Expected Number of Deaths', 'A.3 Total Costs', 'A.3_Expected Number of Deaths', 'A.4 Total Costs', 'A.4_Expected Number of Deaths', 'A.5 Total Costs', 'A.5_Expected Number of Deaths', 'Expected Annual Damage', 'Expected Number of Deaths'])"

df_Sobol['A.1 Total Costs'] = A1_Total_Costs_Score_df
df_Sobol['A.1_Expected Number of Deaths'] = A1_Expected_Number_of_Deaths_Score_df

df_Sobol['A.2 Total Costs'] = A2_Total_Costs_Score_df
df_Sobol['A.2_Expected Number of Deaths'] = A2_Expected_Number_of_Deaths_Score_df

df_Sobol['A.3 Total Costs'] = A3_Total_Costs_Score_df
df_Sobol['A.3_Expected Number of Deaths'] = A3_Expected_Number_of_Deaths_Score_df

df_Sobol['A.4 Total Costs'] = A4_Total_Costs_Score_df
df_Sobol['A.4_Expected Number of Deaths'] = A4_Expected_Number_of_Deaths_Score_df

df_Sobol['A.5 Total Costs'] = A5_Total_Costs_Score_df
df_Sobol['A.5_Expected Number of Deaths'] = A5_Expected_Number_of_Deaths_Score_df

df_Sobol['Expected Annual Damage'] = Expected_Annual_Damage_Score_df[['ST']]
df_Sobol['Expected Number of Deaths'] = Expected_Number_of_Deaths_Score_df[['ST']]"

    

In [ ]:
### 2.5 Creating a visualisation of the results"

ax = sns.heatmap(df_Sobol.T, cmap='viridis', annot=True)

ax.set_aspect('equal')
fig = plt.gcf()
fig.set_size_inches(30,9)

plt.yticks(rotation=0)

plt.show()

In [ ]:
    # 3. Optimalisation"

### 3.1. Searching for candidate solutions with MORO

#To come up with a good policy first we need to see which possible solution are in a costefficient way improving the safety (lower expected yearly damage and yearly deaths because of floodings). The investment costs should ofcourse also be as low as possible."

if __name__ == '__main__':
   ema_logging.log_to_stderr(ema_logging.INFO)

   dike_model, planning_steps = get_model_for_problem_formulation(1)    

   reference_values = {'Bmax': 175, 'Brate': 1.5, 'pfail': 0.5,
                       'discount rate': 3.5,
                       'ID flood wave shape': 4}
   scen1 = {}


   #### Credits for solving namesplit problem and sharing this fix with us to group \"Vera\
   for key in dike_model.uncertainties:
       name_split = key.name.split('_')
       name_split2 = ''.join([i for i in key.name if not i.isdigit()]).rstrip().split('_')

       if len(name_split) == 1:
           scen1.update({key.name: reference_values[name_split2[0]]})

       else:
           scen1.update({key.name: reference_values[name_split[1]]})

   ref_scenario = Scenario('reference', **scen1)

   espilon = [100000,100000,0.1] ### Epsilon values are based on the scales of the output parameters

   convergence_metrics = [HyperVolume(minimum=[0,0,0], maximum=[3e+08, 2.5e+08,0.4]),  ### Numbers selected with generated of earlier optimalisations
                      EpsilonProgress()]

   nfe = 25000    ### 25.000                                                                    ### Witch 25.000 runs convergence is almost perfect

   with MultiprocessingEvaluator(dike_model) as evaluator:
       results_opt, convergence = evaluator.optimize(nfe=nfe,
                                                 searchover='levers',
                                                 epsilons=espilon,
                                                 convergence=convergence_metrics,
                                                 reference=ref_scenario)

In [ ]:
### 3.2 Visualisation of the MORO convergence KPI's"

fig, (ax1, ax2) = plt.subplots(ncols=2, sharex=True, figsize=(8,4))
ax1.plot(convergence.nfe, convergence.epsilon_progress)
ax1.set_ylabel('$\\epsilon$-progress')
ax2.plot(convergence.nfe, convergence.hypervolume)
ax2.set_ylabel('hypervolume')
ax1.set_xlabel('number of function evaluations')
ax2.set_xlabel('number of function evaluations')
plt.show()

    

In [ ]:
### 3.3. Visualisation of the performance of the canidate solutions on the KPI's"

data = results_opt.loc[:, [o.name for o in dike_model.outcomes]]
limits = parcoords.get_limits(data)
print(limits)
limits.loc[0, ['Expected Annual Damage', 'Total Investment Costs', 'Expected Number of Deaths']] = 0

paraxes = parcoords.ParallelAxes(limits)
paraxes.plot(data)

### Because lower scores are better we change the axis for all parameters
paraxes.invert_axis(['Expected Annual Damage', 'Total Investment Costs' , 'Expected Number of Deaths'])
plt.show()

outcomes_opt = results_opt.loc[:, ['Expected Annual Damage', 'Total Investment Costs', 'Expected Number of Deaths']]

fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(outcomes_opt['Expected Annual Damage'], outcomes_opt['Total Investment Costs'], outcomes_opt['Expected Number of Deaths'])
ax.set_xlabel('Expected Annual Damage')
ax.set_ylabel('Total Investment Costs')
ax.set_zlabel('Expected Number of Deaths')
plt.show()

### 3.4. Evaluating canidate solutions under uncertainty

### Which policies options are in the canidate set?
policies = results_opt
policies = policies.drop([o.name for o in dike_model.outcomes], axis=1)
#policies

### Save the policies
policies.to_excel('policies.xls')

In [ ]:
### 3.5. Making a selection of policies to evaluate under uncertainty"

### Makes a distinction between RfR policies and DH only policies
policies['RfR_SUM'] = policies.loc[:,'0_RfR 0':'4_RfR 2'].sum(axis = 1)
RfR = policies[policies['RfR_SUM'] >= 1]
DH = policies[policies['RfR_SUM'] == 0]"

### We are only interested in policies  with less than 0.01 expected deaths (1 expected death every 100 year). 
logical = results_opt['Expected Number of Deaths'] < 0.01
np.sum(logical) ### How many policy options are still valid?"

### Show RfR policies to evaluate
RfR[logical]

### Show DH policies to evaluate
DH[logical]

### Policies to evaluate (Both DH and RfR)
policies_exp = policies[logical]

#policies_exp

policies_exp.to_excel('policies_to_evaluate.xls')

policies_to_evaluate = []

for i, policy in policies_exp.iterrows():
    policies_to_evaluate.append(Policy(str(i), **policy.to_dict()))

In [ ]:
### 3.6. Running the policies under uncertainty"

# Choose problem formuation
dike_model, planning_steps = get_model_for_problem_formulation(2)"

### Selecting number of scenarios to run the policies with
n_scenarios = 2000 ### 2000

with MultiprocessingEvaluator(dike_model) as evaluator:
    results_robust = evaluator.perform_experiments(n_scenarios,
                                           policies_to_evaluate)"

### saving the results
save_results(results_robust, 'MORDM_reevaluation.tar.gz')"

results_robust = load_results('MORDM_reevaluation.tar.gz')

experiments_robust, outcomes_robust = results_robust

In [ ]:
### 3.7. Visualising the results"

from ema_workbench.analysis import pairs_plotting

fig, axes = pairs_plotting.pairs_scatter(experiments_robust, outcomes_robust, group_by='policy',
                                        legend=True)
fig.set_size_inches(20,20)
plt.show()

In [ ]:
### 3.8 How robust are the policies (Calculating regret)

def s_to_n(data, direction):
    mean = np.mean(data)
    std = np.std(data)

    if direction==ScalarOutcome.MAXIMIZE:
        return mean/std
    else:
        return mean*std

In [ ]:
experiments, outcomes = results_robust

overall_scores = {}
for policy in np.unique(experiments['policy']):
    scores = {}

    logical = experiments['policy']==policy

    for outcome in dike_model.outcomes:
        value  = outcomes[outcome.name][logical]
        sn_ratio = s_to_n(value, outcome.kind)
        scores[outcome.name] = sn_ratio
    overall_scores[policy] = scores
scores = pd.DataFrame.from_dict(overall_scores).T
scores

In [ ]:
def calculate_regret(data, best):
   return np.abs(best-data)"

overall_regret = {}
max_regret = {}
for outcome in dike_model.outcomes:
   policy_column = experiments['policy']

   # create a DataFrame with all the relevent information
   # i.e., policy, scenario_id, and scores
   data = pd.DataFrame({outcome.name: outcomes[outcome.name], 
                        \"policy\":experiments['policy'],
                        \"scenario\":experiments['scenario']})

   # reorient the data by indexing with policy and scenario id
   data = data.pivot(index='scenario', columns='policy')

   # flatten the resulting hierarchical index resulting from 
   # pivoting, (might be a nicer solution possible)
   data.columns = data.columns.get_level_values(1)

   # we need to control the broadcasting. 
   # max returns a 1d vector across scenario id. By passing
   # np.newaxis we ensure that the shape is the same as the data
   # next we take the absolute value
   #
   # basically we take the difference of the maximum across 
   # the row and the actual values in the row
   #
   outcome_regret = (data.max(axis=1)[:, np.newaxis] - data).abs()

   overall_regret[outcome.name] = outcome_regret
   max_regret[outcome.name] = outcome_regret.max()


max_regret = pd.DataFrame(max_regret)
fig, ax = plt.subplots(figsize=(10,6))
sns.heatmap(max_regret/max_regret.max(), cmap='viridis', annot=True)
plt.show()

In [ ]:
### 3.9. Visualising the data with Paracoords"

data = scores
limits = parcoords.get_limits(data)
limits.loc[0, ['Expected Annual Damage', 'Dike Investment Costs', 'RfR Investment Costs', 'Evacuation Costs','Expected Number of Deaths']] = 0
#print(limits)
paraxes = parcoords.ParallelAxes(limits, fontsize = 12, rot = 90)
#paraxes.legend()
paraxes.plot(data)
plt.show()

### 3.10. Scenario discovery with PRIM"

x = experiments
y = outcomes['Expected Annual Damage'] < 5e7
prim_alg = prim.Prim(x, y, threshold=0.8, peel_alpha = 0.1)
box3 = prim_alg.find_box(

box3.show_tradeoff()
plt.show()"


box3.inspect(10)
box3.inspect(10, style='graph')
plt.show()"

#box1.select(5)
fig = box3.show_pairs_scatter(10)
#fig.set_size_inches((12,12))
plt.show()

x = experiments
y = outcomes['Expected Number of Deaths'] < 0.005
prim_alg = prim.Prim(x, y, threshold=0.8, peel_alpha = 0.1)
box4 = prim_alg.find_box()


box4.show_tradeoff()
plt.show()"

box4.inspect(11)
box4.inspect(11, style='graph')
plt.show()"

In [ ]:
fig = box4.show_pairs_scatter(11)
#fig.set_size_inches((12,12))
plt.show()"